In [1]:
import yfinance as yf
from pandas_datareader import data as pdr
from datetime import date, timedelta, datetime
import pandas as pd
import threading
import bisect

In [2]:
today = date.today()
day_delta=90

#today = today - timedelta(day_delta)

start = today - timedelta(day_delta)
start= start.strftime("%Y-%m-%d")

end = today + timedelta(1)
end = end.strftime("%Y-%m-%d")
print(start,end)

2020-08-21 2020-11-20


In [3]:
    ticker='SE'
    today = datetime.today()
    days45 = today + timedelta(45)

    optTicker = yf.Ticker(ticker)
    last_price=optTicker.history().tail(1)['Close'].iloc[0]

    date_list=optTicker.options
    date_df = pd.to_datetime(date_list)
    date_diffs = abs(days45 - date_df)
    closest_date = date_list[date_diffs.argmin()]


    opt = optTicker.option_chain(closest_date)

    optCalls=opt[0]
    optPuts=opt[1]

    strikes=optPuts['strike'].tolist()
    strikes.sort()
    
    index = bisect.bisect(strikes, last_price)
    sellPut=strikes[index-1]
    if sellPut-5 in strikes:
        buyPut=sellPut-5
    else:
        index = bisect.bisect(strikes, sellPut-1.5)
        buyPut=strikes[index-1]

    sellDF=optPuts[optPuts['strike']==sellPut]
    buyDF=optPuts[optPuts['strike']==buyPut]

    width=sellDF.strike.iloc[0]-buyDF.strike.iloc[0]
    premium=sellDF.bid.iloc[0]-buyDF.ask.iloc[0]
    if premium<0:
        premium=(sellDF.bid.iloc[0]+sellDF.ask.iloc[0])/2-(buyDF.bid.iloc[0]+buyDF.ask.iloc[0])/2
    if premium>width:
        premium=0
        IV=0
    else:
        IV=sellDF.impliedVolatility.iloc[0]
#     index = bisect.bisect(strikes, last_price)
#     sellPut=strikes[index-1]
#     if sellPut-5 in strikes:
#         buyPut=sellPut-5
#     else:
#         index = bisect.bisect(strikes, sellPut-1.5)
#         buyPut=strikes[index-1]

#     sellDF=optPuts[optPuts['strike']==sellPut]
#     buyDF=optPuts[optPuts['strike']==buyPut]

#     width=sellDF.strike.iloc[0]-buyDF.strike.iloc[0]
#     premium=sellDF.bid.iloc[0]-buyDF.ask.iloc[0]
#     IV=sellDF.impliedVolatility.iloc[0]
    #print(sellPut,buyPut)
    #optPuts.head()
    #print([width,premium,IV])
    #optPuts[(optPuts['strike'] ==67.5) | (optPuts['strike']==62.5) ].head()
    #optPuts.head()

In [4]:
ticker='MSFT'
ahTicker = yf.Ticker(ticker)
ahTicker.history()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-10-19,219.84,221.72,213.16,213.66,27625800,0.00,0
2020-10-20,215.24,216.80,212.53,214.09,22753500,0.00,0
2020-10-21,212.56,216.35,212.56,214.24,22724900,0.00,0
2020-10-22,213.37,215.50,211.15,214.33,22351500,0.00,0
2020-10-23,214.47,215.72,212.60,215.67,18879600,0.00,0
2020-10-26,213.29,215.78,207.56,209.53,37111600,0.00,0
2020-10-27,211.04,214.11,209.78,212.69,36700300,0.00,0
2020-10-28,207.13,208.29,201.57,202.15,51195600,0.00,0
2020-10-29,203.54,206.82,202.84,204.19,31432600,0.00,0


In [5]:
def getPutSpread(last_price,ticker):
    today = datetime.today()
    days45 = today + timedelta(45)

    optTicker = yf.Ticker(ticker)
    #last_price=optTicker.history().tail(1)['Close'].iloc[0]

    date_list=optTicker.options
    date_df = pd.to_datetime(date_list)
    date_diffs = abs(days45 - date_df)
    closest_date = date_list[date_diffs.argmin()]


    opt = optTicker.option_chain(closest_date)

    optCalls=opt[0]
    optPuts=opt[1]

    strikes=optPuts['strike'].tolist()
    strikes.sort()

    index = bisect.bisect(strikes, last_price)
    sellPut=strikes[index-1]
    if sellPut-5 in strikes:
        buyPut=sellPut-5
    else:
        index = bisect.bisect(strikes, sellPut-1.5)
        buyPut=strikes[index-1]

    sellDF=optPuts[optPuts['strike']==sellPut]
    buyDF=optPuts[optPuts['strike']==buyPut]

    width=sellDF.strike.iloc[0]-buyDF.strike.iloc[0]
    premium=sellDF.bid.iloc[0]-buyDF.ask.iloc[0]
    if premium<0:
        premium=(sellDF.bid.iloc[0]+sellDF.ask.iloc[0])/2-(buyDF.bid.iloc[0]+buyDF.ask.iloc[0])/2
    if premium>width:
        premium=0
        IV=0
    else:
        IV=sellDF.impliedVolatility.iloc[0]
    outputArr=[width,premium,IV]
    return outputArr
    #print(optPuts.columns)
    #print (width,premium,IV)




In [6]:
def stok_data_pull():
        dfOutput=pd.DataFrame()

        tickers=['FUTU','GDRX','BKNG','EXPE','MA','ADBE','TSLA','LYFT','PANW','ESTC','SPG','FB','JMIA','TWLO','LULU','LMND','SAGE','ETSY','LRCX','NET','WORK','NFLX','MTB','DKNG','EQIX','EB','PSTH','ARKK','IPOB','TJX','QDEL','DOW','DOCU','CVS','PENN','MGM','MSFT','AMZN','GOOGL','WBA','GDDY','F','BA','GPS','VIAC','MDP','HUYA','ICLN','MJ','PTON','WING','OTRK','FSLY','MELI','SE','CSCO','NNOX','ASAN','AVGO','QCOM','WFC','ISRG','DPZ','PZZA','ROKU','COF','TDOC','CPRI','GRWG','CRM','GLD','SLV','RKT','MDB','TEAM','RDFN','UPS','SQ','LOGI','AMWL','FVRR','TLRY','CRWD','TTD','DDOG','PLAY','U','AMT','MTCH','SDGR','RHP','JPM','SPCE','TTWO','DXCM','VXX','TWOU','RDS-A','BABA']

        for t in tickers:
            df = yf.download(t.replace('.', '-').replace('^', '-'), 
                                  start=start, 
                                  end=end,
                                  prepost=True,
                                  progress=False)

            df['Stock']=t
            dfTemp=df.reset_index()
            last_price=dfTemp[dfTemp['Date']==dfTemp['Date'].max()].Close.iloc[0]
            try:
                putSpread=getPutSpread(last_price,t)
            except:
                putSpread=[0,0,0]
            df['putWidth']=putSpread[0]
            df['putPremium']=putSpread[1]
            df['putIV']=putSpread[2]
            try:
                dfOutput=pd.concat([dfOutput, df])
            except:
                print('error '+t)

        dfOutput.drop_duplicates()
        dfOutput['dt']=''
        dfOutput['dt'].iloc[0] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        dfOutput.to_csv('data.csv')
        return "done"

In [7]:
#one time

stok_data_pull()

'done'

In [8]:
def get_stock_data():
    threading.Timer(300.0, get_stock_data).start() # called every 5 minute
    try:
        stok_data_pull()
    except:
        pass

get_stock_data()